Important Note: using the str.format() method with the {} syntax is somewhat incompatible with writing ImageJ macros. However, the older %s % var syntax is entirely compatible with writing ImageJ macros because the %s syntax has no special meaning in ImageJ macros, unlike the curly braces.

This means that it may be possible to use tkinter or a similar module to create a UI that is straightforward and simple to use

In [18]:
import tkinter.filedialog as filedialog
import glob
import subprocess
from time import sleep
from multiprocessing import Process # this probably is not nessesary
import os
from PIL import Image

In [2]:
def Process_folder_for_analysis(foldername = None):
    if foldername == None:
        foldername = filedialog.askdirectory()
    folder = ''
    for char in foldername:
        if char == '/':
            # backslash is an escape character and escapes itself, so quadruple backslash is required to store two backslashes
            folder += '\\\\'
        else:
            folder += char
    print(folder)
    # Dear future self,
    # The reason past you used the older %s
    # string formatting syntax instead of str.format()
    # was because im ImageJ macros, curly braces are 
    # used and have special meanings
    # %s does not have a special meaning or use in ImageJ
    # macros, which is why you used it instead
    text = '''runMacro("Folder_to_stack.ijm");
run("Image Sequence... ", "format=TIFF name=eXperiment save=%s\\eXperiment0000.tif");''' % folder
    
    with open('C:/Users/Tomas/Desktop/Fiji.app/macros/FolderToFolder.ijm', 'w') as folder_to_folder:
        folder_to_folder.write(text)
    # preprocess.bat starts ImageJ and runs the macro for us
    # this avoids having to type out the file locations again
    !preprocess.bat
    
    return None

The next step is to apply an existing weka model to the dataset. However, weka's use of most of my RAM
compels me to seek a method to apply the model to an image, save the result and close the image, then open the
next image and continue until all images have been processed.

In [3]:
import glob # used to search through the filesystem
def list_files(folder=None):
    if folder == None:
        folder = filedialog.askdirectory()
    folderstring = ''
    # glob.glob and tkinter.filedialog don't agree, so this is needed
    for char in folder:
        # either / or \ is fine, but I would rather only use one.
        # four \ are used because \ is an escape character and I need to end with four \ in the final string dammit
        if char == '/':
            folderstring = '{0}{1}'.format(folderstring, '\\\\') 
        else:
            folderstring = '{0}{1}'.format(folderstring, char)
    
    folder = folderstring 
    files = glob.glob('{0}/*'.format(folder))
    fixed_filenames = []
    for file in files:
        loc = file.rfind('\\') + 1 # +1 for indexing
        fixed_filenames.append(file[0:loc-1]+'\\\\'+file[loc:])
    return fixed_filenames

In [4]:
#list_files()

In [5]:
def weka_macro_input_image(input_name=None):
    
    if input_name == None:
        
        input_name = filedialog.askopenfilename()
    
    input_file = input_name.split('/')[-1]
    
    input_path = ''
    
    for part in input_name.split('/')[:-1]:
        
        input_path = '{0}{1}{2}'.format(input_path, part, '\\\\')
    
    return input_path, input_file

In [6]:
#weka_macro_input_image()

In [7]:
def weka_macro_output_image(output_name=None):
    if output_name == None:
        output_name = filedialog.asksaveasfilename()
    filename = ''
    for char in output_name:
        if char == '/':
            filename += '\\\\'
        else:
            filename += char
    path_pieces = [piece for piece in filename.split('\\\\')[:-1]]
    path = ''
    for part in path_pieces:
        path = '{0}\\\\{1}'.format(path, part)
    path = path[2:]
    name = filename.split('\\\\')[-1]
    return filename, path, name

In [8]:
#weka_macro_output_image()

In [9]:
def write_macro_text(input_path, input_name, output_path, output_name):
    macro_text = '''open("{0}\\\\{1}");
//setTool("freeline");
run("Trainable Weka Segmentation");
wait(3000);
selectWindow("Trainable Weka Segmentation v3.2.14");
call("trainableSegmentation.Weka_Segmentation.loadClassifier", "C:\\\\Users\\\\Tomas\\\\Desktop\\\\Fiji.app\\\\macros\\\\Setaria_classifier_good_enough.model");
call("trainableSegmentation.Weka_Segmentation.applyClassifier", "{0}", "{1}", "showResults=true", "storeResults=false", "probabilityMaps=false", "");
selectWindow("Classification result");
saveAs("Tiff", "{2}\\\\{3}");
'''.format(input_path, input_name, output_path, output_name)
    return macro_text

In [10]:
#print(write_macro_text('C:\\\\Users\\\\Tomas\\\\Desktop\\\\eclipse', 'this.jpg', 'C:\\\\Documents', 'that.jpg'))

In [11]:
def weka_macro_all_images():
    files = list_files()
    
    macros = []
    
    for file in files:
        input_info = weka_macro_input_image(file)
        output_info = weka_macro_output_image(input_info[0]+input_info[1][:-4]+'_ROI.tif')
        macro_text = write_macro_text(input_info[0], input_info[1], output_info[1], output_info[2])
        macros.append(macro_text)
    
    return macros

In [12]:
def run_macro(macro_save_name):
    subprocess.run('C:/Users/Tomas/Desktop/Fiji.app/ImageJ-win64.exe -macro {0} & DEL C:/Users/Tomas/Desktop/Fiji.app/macros/weka_ROI_finding.ijm'.format(macro_save_name))

In [13]:
def weka_image_macros():
    weka_macros = weka_macro_all_images()
    # easier to have a string then asking the user for a macro name every time
    macro_save_name = 'C:/Users/Tomas/Desktop/Fiji.app/macros/weka_ROI_finding.ijm'
    
    for weka in weka_macros:        
        with open(macro_save_name, 'w') as weka_macro:
            weka_macro.write(weka)
        
        run_macro(macro_save_name)
            
def one_macro():
    weka_macros = weka_macro_all_images()
    first_macro = weka_macros[0]
    print(first_macro)
    
    macro_save_name = 'C:/Users/Tomas/Desktop/Fiji.app/macros/weka_ROI_finding.ijm'
    
    with open(macro_save_name, 'w') as weka_macro:  
        weka_macro.write(first_macro)
    
    run_macro(macro_save_name)

In [14]:
#weka_image_macros()

In [15]:
#move the ROI images to a separate folder
def mover():
    folder = filedialog.askdirectory()
    files = glob.glob(folder + '/*_ROI.tif')
    # getting the directory to place the files, and fixing the \ vs / issue
    dest = filedialog.askdirectory()
    dest = dest.replace('/', '\\')
    print(dest)
    for file in files:
    
        # dealing with glob.glob()'s insistence to use a \, as usual
        fileparts = file.split('\\') 
        name = fileparts[0]+'/'+fileparts[1]
        name = name.replace('/', '\\')
    
        runstring = 'MOVE {0} {1}'.format(name, dest)
    
        with open('movefile.bat', 'w') as mover:
            mover.write(runstring)
    
        !movefile.bat
        !DEL movefile.bat

In [16]:
# converting the ROI images to black and white
# this can be skipped, but some ImageJ plugins require a black and white image

from PIL import Image

def roi_image_names(foldername=None):
    if foldername == None:
        foldername = filedialog.askdirectory()
    
    files = glob.glob(foldername+'/*')
    images = []
    for i in range(len(files)):
        files[i] = files[i].replace('\\', '/')
    
    return files

def open_from_names(names):
    images = []
    for name in names:
        images.append(Image.open(name))
    
    return images

def convert_images_to_black_and_white(images):
    # binary is another term used for black and white images
    black_and_whites = []
    for image in images:
        # I turned dithering off because I think it has less noise than dithering
        img = image.convert('1', dither=Image.NONE)
        black_and_whites.append(img)
    
    return black_and_whites

# test case
# no longer needed

#names = roi_image_names()
#images = open_from_names(names)
#black_and_whites = convert_images_to_black_and_white(images)

<b> Next steps </b> 

What I want to do now is figure out:

1. what to measure?
    * of the measurements, which ones can be found from the segmentation?
2. How to measure?
    * how should I go about the specifics of measuring ROIs?
3. Can I integrate the ROIs as a mask on the original grayscale image?
    * Will this help to filder out noise?
    * Can this be used to aquire measurements that wouldn't be obtainable otherwise?
4. Can I filter ROI sizes to reduce false positives?

If I can answer these questions, I will be able to quantify luciferase images effectively.

    

In [14]:
def save_images(image_list, name_base, destination_folder='C:/Users/Tomas/Desktop/bins/'):
    for i in range(len(image_list)):
        image_list[i].save('{0}{1}_{2}.tif'.format(destination_folder, name_base, i))

#save_images(black_and_whites, 'binaryROI')

In [6]:
from PIL import Image
import PIL.ImageOps

def invert_image(imagename):
    # inverts an image
    image = Image.open(imagename)
    image = image.convert('L')
    inverted_image = PIL.ImageOps.invert(image)
    return inverted_image

img = invert_image('C:/Users/Tomas/Desktop/bins/binaryROI_0.tif')
img.show()

In [87]:
def convert_to_array(images_folder='C:/Users/Tomas/Desktop/TimeLabColleen/Luciferase/throw_away_for_inverted_binary_images/', image_num=0):
    from skimage.io import imread
    bins = glob.glob(image_folder + '*')
    nbins = [b.replace('\\', '/') for b in bins]
    example = nbins[image_num]
    im_ex = imread(example)
    return im_ex